# Performance by product --> SoM Dashboard

In [446]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Brands/data/'
CPG = 'hills_science' 
nielsen_cat = False #True JUST FOR RB_HYHO, DANONE, UNILEVER SO FAR!!!
past = True
SOM_CPG = CPG.title()

## Import

In [447]:
import os
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
print([x for x in os.listdir(BASE_DIR) if x.endswith(f'{CPG}.csv')])

['nielsen_hills_science.csv', 'som_hills_science.csv', 'products_hills_science.csv', 'pasillos_hills_science.csv']


In [448]:
df = pd.read_csv(os.path.join(BASE_DIR,f'products_{CPG}.csv'),
                 sep='\t', encoding='UTF-16', low_memory=False)
df.sample(2)

,month,country,city name,store_name,category en,category_id,product id,product name,barcodes,package,...,sales,orders,avg ticket,users,units found,units replaced,units requested,found rate,fulfillment,frequency
20702,agosto 2021,MX,Puebla,Petco,Wet food,227,1109712,Alimento natural estofado de pavo,{653341308047},Lata 340 g,...,200,1,200,1,2.00,0,2,100.0%,100.0%,1.00
13500,agosto 2021,MX,Guadalajara,Soriana,Pet care & grooming,228,1440265,Shampoo para perro,{37836007279},400 ml,...,92,2,46,2,2.00,0,2,100.0%,100.0%,1.00


In [449]:
# df.rename(columns={'month  / year':'month  ','sales [currency]':'sales'}, inplace=True)

In [450]:
aux_ini = df.copy()
aux_ini['sales'] = aux_ini['sales'].astype(str).str.replace(',','').astype(float)
aux_ini = aux_ini.groupby('month  ')['sales'].sum().reset_index()
aux_ini

,month,sales
0,agosto 2021,18115899.00


In [451]:
import re
nielsen = pd.read_csv(BASE_DIR + f'nielsen_{CPG}.csv',
                      sep='\t',encoding='utf-16'
                     ).drop_duplicates(subset=['UPC','PRODUCT ID'])
display(nielsen['UPC'].head())
nielsen['UPC'] = nielsen['UPC'].apply(lambda x: re.sub('[^\d]','',str(x)))
nielsen['UPC'] = nielsen['UPC'].astype(str).str.strip().str.split().str[0].fillna(0).astype(int)
display(nielsen['UPC'].head())
display(nielsen['UPC'].head())
nielsen.sample(2)

0    7502281121649
1    7501072200723
2    7501777001151
3    7501072200730
4    7501072207098
Name: UPC, dtype: object

0    7502281121649
1    7501072200723
2    7501777001151
3    7501072200730
4    7501072207098
Name: UPC, dtype: int64

0    7502281121649
1    7501072200723
2    7501777001151
3    7501072200730
4    7501072207098
Name: UPC, dtype: int64

,UPC,FORMATO,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,PRODUCT ID,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
735,7502234860847,NaN,Bel-Ara,NaN,NaN,Bel Ara,250 g,Maiz enchilado,370332,"Nuts, Seeds and Dried Fruits",NaN,NaN,NaN
349,7501072212764,NaN,Purina MX,NaN,NaN,Beneful,Sobre 100 g,Alimento húmedo para perro adulto sabor pollo,1250708,Wet food,NaN,NaN,NaN


## Functions

In [452]:
import time
import numpy as np
from IPython.lib.display import Audio

start = time.time()
def time_exp(x):
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

In [453]:
def limpio(data,month_col,to_drop):
    df = data.copy()
    df[['mes','anio']] = df[month_col].str.split(expand=True)
    meses = dict(zip(['enero','febrero','marzo','abril','mayo','junio','julio',
                      'agosto','septiembre','octubre','noviembre','diciembre'],range(1,13)))
    df['mes'] = df['mes'].replace(meses)
    df.drop(columns=to_drop,inplace=True)
    df.columns = [str(x).replace('.','').replace(' ','_').lower() for x in df.columns]
    for col in ['brand_filter','cpg_filter']:
        df[f'{col[:-7]}_id'] = df[col].apply(lambda x: ','.join(re.findall('\s\((\d+)\)',str(x))))
        df[f'{col[:-7]}_name'] = df[col].apply(lambda x: re.sub('\s\(\d+\)','',str(x)))
    return df

## Transform

### Barcodes

In [454]:
df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')
df['barcodes'] = df['barcodes'].str.split(',',n=1).fillna('.').apply(lambda x:x[0])
df['barcodes'].sample(4)

/var/folders/j4/1p33_bh96yn8pdz3b_5t80hm0000gn/T/ipykernel_1740/2845833164.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')


8415      800443285183
22439     792763002017
19292    7501072212665
3080                 .
Name: barcodes, dtype: object

### Merge

In [455]:
df = df.astype(str).merge(nielsen.astype(str).rename(columns={'UPC':'barcodes',
                                                              'PRODUCT ID':'product id'}),
                          on=['barcodes'] if nielsen_cat else ['barcodes','product id'],
                          how='left' if nielsen_cat else 'left')
if nielsen_cat:
    df = df.rename(columns={'product id_x':'product_id'}).drop(columns='product id_y')

df.sample(2)

,month,country,city name,store_name,category en,category_id,product id,product name,barcodes,package,...,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
14330,agosto 2021,MX,León,HEB,"Nuts, Seeds and Dried Fruits",1004,335231,Cacahuate salado,7501011124608,180 g,...,PepsiCo MX,nan,nan,Mafer,180 g,Cacahuate salado,"Nuts, Seeds and Dried Fruits",nan,nan,nan
1065,agosto 2021,MX,Cancún,Costco,"Nuts, Seeds and Dried Fruits",1004,152333,Almendras tostadas,96619723829,1.13 kg,...,Kirkland,nan,nan,Kirkland Signature,1.13 kg,Almendras tostadas,"Nuts, Seeds and Dried Fruits",nan,nan,nan


### CPG

In [456]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
agosto 2021,18115899.00,18115899.00,0.00,0.00


In [457]:
prod_cpg = pd.read_csv(os.path.join(BASE_DIR,'pbbc.csv'),
                       #sep='\t',encoding='utf-16'
                      )
prod_cpg.sample(4)

,n,product_id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name
32193,1,219485,30111416735,230,Alimento seco,7189,Royal Canin,87,Royal Canin
108321,1,3493655,7503032498478,1010,Pavo,73506,YEMA,706,Yema
107673,1,3346294,7501044022667,244,Electrodomésticos,6297,Vasconia,757,Vasconia
39900,1,314173,7501058619976,23,Sin lactosa,5065,La Lechera,3,Nestlé MX


In [458]:
prod_cpg.drop_duplicates(('barcodes' if nielsen_cat else 'product_id'),inplace=True)
prod_cpg = prod_cpg.astype(str).rename(columns={'product_id':'product id'})
prod_cpg['cpg filter'] = prod_cpg['cpg_name']+" ("+prod_cpg['cpg_id'].astype(str)+")"
prod_cpg.sample(4)

,n,product id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name,cpg filter
83258,1,1246585,75079215,679,Sexo seguro,8854,Pfizer,355,Pfizer,Pfizer (355)
67223,1,596952,72457331051,72,Aderezos para ensaladas,9275,Walden Farms,781,Smart,Smart (781)
87105,1,1447142,13117031157,51,Pañales y toallitas,6896,Kiddies,117,Ontex,Ontex (117)
6152,1,112333,650240025303,978,Accesorios de belleza,7384,Zan Zusi,67,Genomma Lab,Genomma Lab (67)


In [459]:
df = df.merge(prod_cpg[[('barcodes' if nielsen_cat else 'product id'),'cpg filter']],
              on=('barcodes' if nielsen_cat else 'product id'),how='left')

In [460]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
agosto 2021,18115899.00,18115899.00,0.00,0.00


### Limpio

In [461]:
df = limpio(df,'month  ',
            ['month  ','country ','avg ticket','found rate','fulfillment','frequency'])
df.sample(2)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id,cpg_name
25600,Toluca,Petco,Dry food,230,1474020,Alimento para perro adulto pequeño,30111451040,Costal 1.6 kg,nan,Royal Canin (7189),...,nan,nan,nan,Mars MX Pet (1271),8,2021,7189,Royal Canin,1271,Mars MX Pet
23859,San Luis Potosí,HEB,"Nuts, Seeds and Dried Fruits",1004,338000,Cacahuates Kiyakis,7501030420583,150 g,nan,Barcel (3903),...,nan,nan,nan,Grupo Bimbo (36),8,2021,3903,Barcel,36,Grupo Bimbo


In [462]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
ago,18115899.00,18115899.00,0.00,0.00


In [463]:
aux = df[df['formato'].astype(str)=='nan'].copy()
df = df[~(df['formato'].astype(str)=='nan')].copy()
null_fill = {'proveedor_general':'cpg_name',
             'proveedor_abierto':'cpg_name',
             'marca_detalle':'brand_name',
             'marca_generica':'brand_name',
             'gramaje':'package',
             'descripcion_nielsen':'product_name',
             'segmento':'category_en'}
for x,y in null_fill.items():
    aux.loc[:,x] = aux.loc[:,y]
    
df = df.append(aux,ignore_index=True)

In [464]:
for col in df.columns:
    df[col] = df[col].astype(str).str.replace('nan','')
    
df = df.drop(columns='cpg_name').fillna('')
df.sample()

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,segmento,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id
1115,Cancún,OXXO,Chips & Snacks,996,333298,Torciditos frituras sabor queso y chile,7501011179417,255 g,,Cheetos (4185),...,Chips & Snacks,,,,PepsiCo MX (2),8,2021,4185,Cheetos,2


In [465]:
brand_cols = ['formato','proveedor_general','proveedor_abierto','marca_detalle','marca_generica','gramaje','descripcion_nielsen','segmento','subsegmento_1','subsegmento_2','subsegmento_3']
display(df[brand_cols].sample())

for col in brand_cols:
    df[col] = df[col].str.strip().str.title()

display(df[brand_cols].sample())

,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
13381,,Purina MX,Purina MX,Dog Chow,Dog Chow,2 kg,Alimento seco Extralife cachorros minis y pequ...,Dry food,,,


,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
14558,,,,Merrick,Merrick,Costal 9.9 Kg,Alimento Perro Libre De Granos,Dry Food,,,


In [466]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux_fin

,mes,sales
0,ago,18115899.00


### Marca genérica

In [467]:
df.loc[df[df['marca_generica']==''].index, 'marca_generica'] = df.loc[df[df['marca_generica']==''].index, 'marca_detalle']
df[df['marca_generica']=='']

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,segmento,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id


## Export

### Same product_id as past

In [468]:
print(len(df))
if past:
    som = pd.ExcelFile(os.path.join('/'.join(BASE_DIR.split('/')[:-2]),
                                    f'SoM_{SOM_CPG}.xlsx')).parse(sheet_name=0)
    df = df.merge(som.drop_duplicates('product_id')[['product_id']].reset_index(drop=True).astype(str))
    
print(len(df))

25880
14076


In [469]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))

In [470]:
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1

### Ordered columns

In [471]:
export = df[['city_name', 'store_name', 'category_en', 'category_id', 'product_id',
             'product_name', 'barcodes', 'package', 'product_sku', 'brand_name',
             'sales', 'orders', 'users', 'units_found', 'units_replaced',
             'units_requested', 'formato', 'proveedor_general', 'proveedor_abierto',
             'marca_detalle', 'marca_generica', 'gramaje', 'descripcion_nielsen',
             'segmento', 'subsegmento_1', 'subsegmento_2', 'subsegmento_3',
             'mes', 'anio']].copy()
export.sort_values(['anio','mes'],inplace=True)
export.sample(4)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_name,...,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3,mes,anio
1291,Toluca,Soriana,"Nuts, Seeds and Dried Fruits",1004,335235,Cacahuate surtido especial salado,7501011125292,Bolsa 180 g,,Mafer,...,Mafer,Mafer,Bolsa 180 G,Cacahuate Surtido Especial Salado,"Nuts, Seeds And Dried Fruits",,,,8,2021
5225,Ciudad de México,Soriana,Wet food,227,449695,Alimento húmedo para perro adulto raza pequeña...,7501072210234,100 g,,Dog Chow,...,Dog Chow,Dog Chow,100 G,Alimento Húmedo Para Perro Adulto Raza Pequeña...,Wet Food,,,,8,2021
10161,Monterrey,HEB,"Nuts, Seeds and Dried Fruits",1004,1004186,Nuez y dátil,7500327047830,50 g,,Okko Super Foods,...,Okko Super Foods,Okko Super Foods,50 G,Nuez Y Dátil,"Nuts, Seeds And Dried Fruits",,,,8,2021
11433,Querétaro,Petco,Dry food,224,392093,All life stages alimento natural para gato rec...,800443247969,1.1 kg,125330,Whole-hearted,...,Whole-Hearted,Whole-Hearted,1.1 Kg,All Life Stages Alimento Natural Para Gato Rec...,Dry Food,,,,8,2021


In [472]:
export.to_csv(os.path.join(BASE_DIR,f'som_{CPG}.csv'),index=False,sep='\t',encoding='utf-16')

## Fin

In [473]:
aux

,sales,sales_fin,dif,dif_p
month,,,,
ago,18115899.00,12569550.00,-5546349.00,-0.31


In [474]:
time_exp(time.time() - start)
tono()

0 minutos con 26.20 segundos
